In [17]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M
from scipy.optimize import fsolve

# Simple, conceptual model of marginal sea

In [18]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6          # [m] zonal extent 
L_y = 2e6          # [m] meridional extent 
A = L_x*L_y        # [m^2] surface area of interior 
P = 2*L_x+2*L_y    # [m] Perimeter 

g = 9.81           # [m/s^2] gravitational acceleration 
C_p = 4.2e3        # [J/(kg K)] specific heat capacity of sea water 
c = 0.006          # [] bottom slope coefficient
s = 0.02           # [] average bottom slope around the perimeter of the marginal sea

#### Parameters ####
H = 1000           # [m] sill height 
f_0 = 1.2e-4       # [s^-1] Coriolis parameter 
Gamma = 20         # [W/(m^2 C)] Restoring strength 
alpha_T = 0.2      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
T_1 = 8.87         # [Celsius] ingoing temperature 
T_A = 2.8          # [Celsius] Atmospheric temperature 
ro = 1027          # [kg m^-3] density sea water 
S = 35             # [ppt] net evaporation minus precipitation
S_1 = 35.2         # [g kg^-1] incoming salinity; source Lambert                          ! is ppt = g kg^-1 ???
E = -2e-8          # [m/s] a representative ocean salinity; source: Spall[2015]


#### functions ####
L = H/s           # [m] width of sloping topography of the boundary current 


In [34]:
############# Computation ##################
############################################

# Main equations

def V1(T1,T0):           # depth-averaged baroclinic velocity in the boundary current
    (alpha_T*g/(2*ro_0*f_0))*((T1-T0)*H/L)     #only dependent on T

def eddy(V,X1,X0):       # eddy flux is proportional to V1 and the gradient between the parameters
    c*V*(X1-X0)          # for salt (S) and heat (T) fluxes

def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
    return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0-T_A),
            P*H*x[6]+A*E*S,
            (T_1-x[1])*x[4]*H*L+((1+P*L/A)*(T_1-T_A)-(T_1-x[0]))*A*Gamma/(ro_0*C_p),
            (S_1-x[3])*x[4]*H*L+E*A*S,
            (alpha_T*g/(2*ro_0*f_0))*((T_1-x[0])*H/L)-x[4],
            c*x[4]*(T_1-x[0])-x[5],
            c*x[4]*(S_1-x[2])-x[6]]

In [35]:
root = fsolve(main, [1,1,1,1,1,1,1])
root

array([2.80539733e+00, 9.04594113e+00, 3.51597213e+01, 3.51709993e+01,
       9.65494193e-01, 3.51320319e-02, 2.33333333e-04])

In [3]:
# Since we're not getting acceptable results for T_out and T_0
# try to fix some parameters and use values from the literature

mu = 0.15*epsilon # value for mu/epsilon retrieved from table 1 in Spall frist row
print('mu/epsilon =', mu/epsilon)

# calculate temperatures with fixed mu
T_0 = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)
T_out = T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)
print('T_1=', T_1, ', T_out=', T_out)


mu/epsilon = 0.15
T_1= 8.87 , T_out= 6.573029381034166


In [6]:
# Now that we have workable values for the temperatures 
# we can try to find values for the paramters that are clearly not working

# alpha is clearly incorrect, by fixing W we can check it's value
W = 2.32e6
Delta = (T_out-T_0)/(T_1-T_0)
alpha = W*(4*ro_0*f_0)/(g*H**2*(T_1-T_0)*(1-Delta))
print('alpha=', alpha)

alpha= 0.050754577676224044


In [11]:
#### Two main equations #####

#Interior temperature [Celsius]
T_0 = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)

#Outflowing temperature [Celsius]
T_out = T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)

(6.333402634099233, 6.573029381034166)

In [16]:
#### Follow up computations ####

#Velocity in boundary current [m/s]
V_1 = alpha*g*H*(T_1-T_0)/(2*ro_0*f_0*L)

#effective perimeter [m]
#P_e = ... #[m] -> Not sure if we have to use this, and then how to determine it?

#Atmospheric heat flux/meridional heat transport at the sill [W/m^2]
Q_star = A*(T_0 - T_A)*Gamma + P*L*Gamma*(T_1-T_A)

#Fraction of outflowing waters that include inflowing water [-]
Delta = (T_out-T_0)/(T_1-T_0)

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = alpha*(g*H**2*(T_1-T_0)*(1-Delta))/(4*ro_0*f_0)


In [15]:
print('V_1=', "{:.2e}".format(V_1), ', Q_star=', "{:.2e}".format(Q_star), ', W=', "{:.2e}".format(W))

V_1= 1.02e-01 , Q_star= 1.78e+14 , W= 2.32e+06
